# Brain-Tumor-Image-Classification-Machine-Learning-Project

In this ML project we will create an ML model that can detect and classify brain tumors by image recognition. We will use scanned MRIs of healthy and unhealthy brains to train the model. By using the Image classification our model will be able to classify brain scans as Helathy and Unhealthy.

We have 4600 scanned images of healthy and unhealthy brains sectioned into two folders Healthy and Unhealthy, Our model will have to classify each scan to respective Tumor and No Tumor categories.

The dataset is from Kaggle's brain-tumor-dataset.

We will use CNNs for model building by making use of Binary Image Classification

Binary Image Classification: Binary classification involves classifying images into one of two categories. For example, determining whether an image contains a cat or not in our case Healthy & Unhealthy brain. This is the simplest form of image classification.

For the classifiaction algorithm we will use the TensorFlow platform and Keras API

# Import relevant libraries

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import keras
import tensorflow as tf


In [2]:
from keras.preprocessing.image import load_img # reading the image
from tensorflow.keras.preprocessing.image import ImageDataGenerator # reads the image from database 
# tf.config.experimental_run_functions_eagerly(True)
# tf.data.experimental.enable_debug_mode(True)
tf.data.experimental.enable_debug_mode()
tf.config.run_functions_eagerly(True)

In [ ]:
img_dir = "/Users/asus/Documents/DS/## 180 Projects/PROJECTS/Brain tumor project/hands/Brain Tumor Data Set/Brain Tumor Data Set"

In [ ]:
BATCH_SIZE = 64
IMAGE_SIZE = 150
input_shape = (150,150,1) # shape of the image

In [ ]:
data_gen = ImageDataGenerator(rescale=1./255,
                             validation_split=0.2) #Normalizing the image


In [ ]:
# Training Dataset
train_gen = data_gen.flow_from_directory(img_dir,
                                        target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                        batch_size= BATCH_SIZE,
                                        color_mode = "grayscale",
                                        shuffle=True,
                                        class_mode = "binary",
                                        subset= "training")

In [ ]:
# Validation Dataset
val_gen = data_gen.flow_from_directory(img_dir,
                                      target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                      batch_size=BATCH_SIZE,
                                      color_mode = "grayscale",
                                      shuffle=False,
                                       class_mode = "binary",
                                       subset="validation"
                                      )

In [ ]:
labels = train_gen.class_indices #(an index allowing documents to be indexed, deleted, and searched)
classes = list(labels.keys())

print(classes)

## Modelling

For the modelling purposes we will use a convolutional neural network

## Convolutional Neural Networks.

* CNNs are very similar to ordinary Neural Networks, they are made up of neurons that have learnable weights and biases. Each neuron receives some input, performs a dot product and optionally follows it with a non-linearity.

* CNNs forward propagation efficient and vastly reduces the number of parameters in the network

### Layers

* Input layer will hold the raw pixel values of the image, having the shape of (W,H,D).D denotes depth whether RGB(3), Grayscale(1),etc.

* CONV layer will compute the output of neurons that are connected to local regions in the input, each computing a dot product b/w their weights and a small region.

* RELU is the activation function, The rectified linear unit (ReLU) or rectifier activation function introduces the property of nonlinearity to a deep learning model and solves the vanishing gradients issue. It interprets the positive part of its argument.

* POOLING layer will perform a down-sampling operation along the spatial dimensions(W,H)

* Fully connected layer will compute the class scores as in a regular neural network, As with ordinary Neural Networks and as the name implies, each neuron in this layer will be connected to all the numbers in the previous volume.

# Implementing CNN

* We will use TensorFlow and the Keras API for implementation
* We will add 2 layers of convolutional layers along with 2 Max Pooling layers.
* Our dense layer will have 512 neurons, Our input layer wil be defining the size of image being fed into the network and an output layer will be having one neuron since it is a classifiaction problem
* Saving the model and deployment of Flask Webapp

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dropout

In [ ]:
model =  Sequential()

model.add(keras.layers.InputLayer(input_shape = (150,150,1)))
model.add(Conv2D(16,(3,3), activation = "relu"))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(32,(3,3), activation = "relu"))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dense(512, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation = "sigmoid"))

In [ ]:
model.compile(optimizer="adam",
             loss = "binary_crossentropy",
             metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
with tf.device('/device:GPU:0'): # using the GPU
    history = model.fit(train_gen,
                        verbose = 1,
                        epochs=5,
                        validation_data= val_gen,
                        steps_per_epoch= 3681//64,
                        validation_steps = 919//64)

In [ ]:
test_loss, test_acc =  model.evaluate(val_gen)

In [ ]:
loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, "y", label = "Training loss")
plt.plot(epochs, val_loss, 'r', label = "Validation loss")

plt.title('Training and validation loss')
plt.xlabel("Epochs")
plt.ylabel('Loss')

plt.legend()
plt.show()


acc =  history.history["accuracy"]
val_acc = history.history["val_accuracy"]
plt.plot(epochs, acc, 'y', label = "Training acc")
plt.plot(epochs, val_acc, 'r', label = "Validation acc")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")

plt.legend()
plt.show()

In [ ]:
# model.save("model.h5")
model.save("model.keras")

In [ ]:
from matplotlib.pyplot import imshow
from PIL import Image, ImageOps

data = np.ndarray(shape = (1,150,150,1), dtype = np.float32)
image= Image.open(f'/Users/asus/Documents/DS/## 180 Projects/PROJECTS/Brain tumor project/hands/Brain Tumor Data Set/Brain Tumor Data Set/Healthy/Not Cancer  (39).jpg')
#image = Image.open(f'/Users/asus/Documents/DS/## 180 Projects/PROJECTS/Brain tumor project/hands/Brain Tumor Data Set/Brain Tumor Data Set/Brain Tumor/Cancer (25).tif')
size = (150,150)
image = ImageOps.grayscale(image)
image = ImageOps.fit(image, size)
image_array = np.asarray(image)
display(image)

# normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
data = image_array.reshape((-1,150,150,1))

#data[0] = normalized_image_array

In [ ]:
prediction =  model.predict(data)
print(prediction[0][0])